## TIMORIS: Happy Path Tutorial
This notebook demonstrates the minimal working pipeline for detecting tubule-like structures in spatial transcriptomics data using the TIMORIS tool.

## Step 1: Load and Preprocess Data

In [12]:
import numpy as np
import networkx as nx
import scanpy as sc
import scipy.sparse as sp
from scipy.spatial import Delaunay
from sklearn.neighbors import kneighbors_graph, radius_neighbors_graph
import scipy.sparse as sp
from scipy.sparse import csr_matrix
import community as community_louvain
import igraph as ig
import leidenalg
from sklearn.cluster import KMeans
from scipy.linalg import eigh


In [1]:
from data_preprocessing import preprocess_data

# Path to your input file (update as needed)
input_path = 'example_data/example_dataset.h5ad'
adata = preprocess_data(input_path, file_type='h5ad')
adata

/home/shuzh/.local/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


View of AnnData object with n_obs × n_vars = 111029 × 491
    obs: 'label', 'vol', 'p0', 'p1', 'z', 'max_corr', 'ncounts', 'gc_12clust', 'tube', 'ang', 'rad', 'dist_to_edge', 'abin_maxcorr', 'abin_maxcorr_no1', 'good_area', 'sert_abin_maxcorr', 'kmeans12', 'n_genes'
    var: 'gene_symbols', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'

In [2]:
adata.obsm['spatial'] = adata.obs[['p1', 'p0']].to_numpy()

/tmp/ipykernel_1830215/720783795.py:1: ImplicitModificationWarning: Setting element `.obsm['spatial']` of view, initializing view as actual.
  adata.obsm['spatial'] = adata.obs[['p1', 'p0']].to_numpy()


## Step 2: Generate Spatial Graph

In [3]:
from spatial_graph import generate_spatial_graph

adata = generate_spatial_graph(adata, method='knn', k=6)
adata.obsp['spatial_connectivities'][:5, :5]  # Show part of the adjacency matrix

<5x5 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [4]:
adata

AnnData object with n_obs × n_vars = 111029 × 491
    obs: 'label', 'vol', 'p0', 'p1', 'z', 'max_corr', 'ncounts', 'gc_12clust', 'tube', 'ang', 'rad', 'dist_to_edge', 'abin_maxcorr', 'abin_maxcorr_no1', 'good_area', 'sert_abin_maxcorr', 'kmeans12', 'n_genes'
    var: 'gene_symbols', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'log1p', 'hvg'
    obsm: 'spatial'
    obsp: 'spatial_connectivities'

In [5]:
adata.obsp['spatial_connectivities']

<111029x111029 sparse matrix of type '<class 'numpy.float64'>'
	with 666174 stored elements in Compressed Sparse Row format>

### Step 3: Load Graph and Run Clustering

In [ ]:
from spatial_graph import load_spatial_graph, cluster_spatial_graph

In [22]:
def cluster_spatial_graph(graph, method='leiden'):

    """
    Perform graph-based clustering using the specified method.
    Supported methods: 'louvain', 'leiden', 'spectral'.
    """

    if method == "louvain":
        cluster_map = community_louvain.best_partition(graph)
    elif method == "leiden":
        # Convert networkx to igraph
        node_list = list(graph.nodes)
        node_index = {node: idx for idx, node in enumerate(node_list)}
        ig_graph = ig.Graph()
        ig_graph.add_vertices(len(node_list))
        ig_graph.add_edges([(node_index[u], node_index[v]) for u, v in graph.edges])

        # Perform Leiden clustering
        partition = leidenalg.find_partition(ig_graph, leidenalg.ModularityVertexPartition)

        # Map back cluster assignments
        cluster_map = {node_list[i]: cluster_id for i, cluster_id in enumerate(partition)}
    elif method == "spectral":
        # compute Laplacian matrix
        A = nx.adjacency_matrix(graph).toarray()
        D = np.diag(A.sum(axis=1))
        L = D - A
        # get eigenvectors
        _, eigvecs = eigh(L, subset_by_index=[1, 3]) # filler for now: ask Shu what he would want num_clusters to be
        # apply k-means clustering
        kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
        labels = kmeans.fit_predict(eigvecs)
        cluster_map = {node: labels[i] for i, node in enumerate(graph.nodes())}
    else:
        raise ValueError("Unsupported clustering method")

    # Assign cluster labels to graph nodes
    for node, cluster in cluster_map.items():
        graph.nodes[node]["cluster"] = cluster

    return graph

In [13]:
# Save to temporary file for graph loading demo (if needed)
# adata.write('temp_output.h5ad')
graph = load_spatial_graph(adata)

In [25]:
graph = cluster_spatial_graph(graph, method='leiden')
